In [1]:
#from tweepy import OAuthHandler
#from tweepy.streaming import StreamListener
import tweepy as tw
import json
import pandas as pd
import numpy as np
#import cs
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time

In [2]:
consumer_key = "#####################################"
consumer_secret = "###############################"
access_key = "##########################################"
access_secret = "#####################################" 
bearer_token = "###############################################"

In [3]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tw.API(auth, wait_on_rate_limit=True)


In [3]:
client = tw.Client(bearer_token, wait_on_rate_limit = True)

In [12]:
sitin_tweets = []
for response in tw.Paginator(client.search_all_tweets, 
                                 query = 'اعتصام_القيادة_العامة# -is:retweet lang:ar',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2019-04-01T00:00:00Z',
                                 end_time = '2019-06-30T00:00:00Z',
                              max_results=500):
    time.sleep(1)
    sitin_tweets.append(response)

Rate limit exceeded. Sleeping for 24 seconds.
Rate limit exceeded. Sleeping for 84 seconds.


In [17]:
mawakip_tweets = []
for response in tw.Paginator(client.search_all_tweets, 
                                 query = 'موكب_31ديسبمر# OR موكب13يناير# OR موكب6يناير# OR موكب9يناير# OR موكب20يناير# OR موكب17يناير# OR موكب24يناير# OR موكب31يناير# -is:retweet lang:ar',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'id' ,'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2018-12-18T00:00:00Z',
                                 end_time = '2019-01-30T00:00:00Z',
                              max_results=500):
    time.sleep(1)
    mawakip_tweets.append(response)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [22]:
tasgotbs_tweets = []
for response in tw.Paginator(client.search_all_tweets, 
                                 query = '(تسقط_بس#)  -is:retweet lang:ar',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'id' ,'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2018-12-19T00:00:00Z',
                                 end_time = '2019-09-30T00:00:00Z',
                                 max_results=500):
    time.sleep(1)
    tasgotbs_tweets.append(response)

Rate limit exceeded. Sleeping for 7 seconds.
Rate limit exceeded. Sleeping for 47 seconds.
Rate limit exceeded. Sleeping for 73 seconds.


In [23]:
result = []
user_dict = {}
# Loop through each response object
for response in tasgotbs_tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'followers': user.public_metrics['followers_count'],
                              'tweets': user.public_metrics['tweet_count'],
                              'description': user.description,
                              'location': user.location
                             }
    for tweet in response.data:
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet.author_id, 
                       'username': author_info['username'],
                       'author_followers': author_info['followers'],
                       'author_tweets': author_info['tweets'],
                       'author_description': author_info['description'],
                       'author_location': author_info['location'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],
                       'quote_count': tweet.public_metrics['quote_count']
                      })

# Change this list of dictionaries into a dataframe
df = pd.DataFrame(result)

In [24]:
df

,author_id,username,author_followers,author_tweets,author_description,author_location,text,created_at,retweets,replies,likes,quote_count
0,186577247,monta_albany,2403,183468,#zamalik #liverpool\nhttps://t.co/VUNseGXJOd\n...,مصر,#السيسي_جاب_ورا\nبس برده #تسقط_بس #ارحل_يا_بلح...,2019-09-29 23:47:50+00:00,0,0,1,0
1,1174026176622448645,aO14iHND8LzqWth,5,64,,None,@AlsisiOfficial تسقط بس,2019-09-29 22:50:51+00:00,0,0,0,0
2,3022421681,TfreedomSD,2295,10879,إن آخر مكان في الجحيم محجوز لمن يقفون على الحي...,None,@hash_sudan حاميها حراميها\n#تنظيف_مفاصل_الدول...,2019-09-29 22:23:39+00:00,0,0,0,0
3,1153787439661690880,OmMarim78133417,57,3022,,None,#الشعب_يريد_إسقاط_الحمار\nاللهم لطفك ورحمتك بأ...,2019-09-29 22:21:53+00:00,0,0,0,0
4,1469163769,dragonswave,1562,43619,#ابن_السودان \nأحد التنانين 🐉🔥 عش كتنين ! \n\n...,None,@Mohamed41127289 @sereza700 لا انا مافارق معاي...,2019-09-29 21:05:35+00:00,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
695137,918411751183536128,Mass_of_pain,141,2914,في حياةٍ ما ، كُنتُ هناك بكاملي ، أما الآن فأن...,Sudan,قالت له :صباح الخير حبيبي ، كيف أصبحت اليوم ؟!...,2018-12-21 11:00:41+00:00,4,0,3,0
695138,918411751183536128,Mass_of_pain,141,2914,في حياةٍ ما ، كُنتُ هناك بكاملي ، أما الآن فأن...,Sudan,قالت له : صباح الخير حبيبي كيف أصبحت اليوم ؟! ...,2018-12-21 10:43:37+00:00,0,0,0,0
695139,1614599150,FatohJaweesh,1030,40194,I See The World in a Different Vision.. 🗺,Everywhere ..,حكومة تحرمني منك تسقط بس ✌️\nاحييي يا اخوانا \...,2018-12-21 10:17:50+00:00,2,0,4,1
695140,2847230308,wely_yahia,872,17698,‏‏‏ زول بيحلم بالعديلة وفرحة الريدة الطويلة...,khartoum,#مدن_السودان_تنتفض\nحكومة تحرمني منك تسقط بس ✌...,2018-12-21 09:36:04+00:00,10,1,22,0


In [25]:
df.to_csv('tasgod.csv', encoding = "utf-8-sig")

In [25]:
tweets = tw.Cursor(api.search_tweets,
              q=search_words,
              lang="en",
              since=date_since).items(100)
tweets

In [26]:
users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in tweets]
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['user', "location"])
tweet_text

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


,user,location
0,RahmanBunairee,"Washington, D.C., Pakhtunkhwa"
1,FLBZH35,"Ille-et-Vilaine, Bretagne"
2,expo_technology,
3,adekleine,
4,Bassomrt,"Aveiro, Portugal"
...,...,...
95,LaurentAlfonso,"Barcelone, Espagne"
96,Eurocentrique,"Brussels, Belgium"
97,CoE_CERIDES,Cyprus
98,robinsnewswire,RT's Are FYI Purposes Only


In [41]:
tweets = tw.Cursor(api.search_tweets,
                   q= "اعتصام_القيادة_العامة#",
                   lang="ar",
                   start_time = "2019-01-01T00:00:00.000Z",
end_time = "2022-06-01T00:00:00.000Z").items(2000)

##all_tweets = [ tweet in tweets]
#all_tweets


In [37]:
users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in tweets]
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['user',"location"])


In [42]:
all_tweets = [ tweet.text for tweet in tweets]


Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpec

Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time
Unexpected parameter: start_time
Unexpected parameter: end_time


In [29]:
users_locs = [[tweet.user.screen_name, tweet.user.location] for tweet in tweets]
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['user',"location"])

In [35]:
all_tweets = [tweet.text for tweet in tweets]
tweet_text = pd.DataFrame(data= all_tweets, 
                    columns= ["tweets_text"])

In [47]:
df = pd.DataFrame(all_tweets)

In [50]:
pd.DataFrame(set(all_tweets))

,0
0,#مليونية3يونيو #مجزرة_القيادة_العامة \nلجان ا...
1,RT @hillagadida: تنسيقيات لجان مقاومة مدينة ال...
2,تخليداً للذكرى الثالثة لفض اعتصام القيادة العا...
3,بداية قمع اعتصام اليوم الواحد في محطة 7 \n\n#م...
4,RT @KordofanRCA: لجان المقاومة _ الأبيض\n٣ يون...
...,...
478,أحيا عشرات الالاف من السودانيين …في العاصمة وا...
479,تنسيقيات لجان مقاومة مدينة الخرطوم \n\nنحتسب ش...
480,RT @KhartoumHashtag: لقاء مهم يجب على الثوار م...
481,RT @AJArabic: في الذكرى الثالثة لفض اعتصام الق...


1000

In [ ]:
# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['text'])
csvFile.close()

def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 10. Tweet text
        text = tweet['text']
        


Unauthorized: 401 Unauthorized
Unauthorized

In [10]:
with open("myfile.txt", "w") as file1:
    for response in tw.Paginator(client.search_all_tweets, 
                                 query = '#BLM -is:retweet lang:en',
                                 user_fields = ['username', 'public_metrics', 'description', 'location'],
                                 tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                 expansions = 'author_id',
                                 start_time = '2022-06-24T00:00:00Z',
                                 end_time = '2022-06-25T00:00:00Z',
                              max_results=500):
        time.sleep(1)
    # Writing data to a file
        file1.write(response.id)
  

AttributeError: 'Response' object has no attribute 'id'